# This is for translating responses in other languages into english to check for errors. 

In [ ]:
from openai import OpenAI

import os

client = OpenAI(api_key="")

def translate(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or "gpt-4" if available
        messages=[
            {"role": "user", "content": f"Translate the following from Portuguese to English:\n\n{text}"}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()


In [ ]:
import json
import random
import pandas as pd
from probe_gen.paths import data

# Load all data
file_path = data.metaphors_brazil / "qwen_3b_brazil_prompted_balanced_1k.jsonl"
with open(file_path, "r", encoding="utf-8") as f:
    lines = [json.loads(line) for line in f]

# Sample 10 random rows
sampled = random.sample(lines, 10)

# Create a clean list of dictionaries for export
rows = []
for i, ex in enumerate(sampled):
    row = {
        "Portuguese Input": ex.get("input", ""),
        "English Input": translate(ex.get("input", "")),
        "Model Output (PT)": ex.get("model_outputs", ""),
        "Model Output (EN)": translate(ex.get("model_outputs", "")),
        "Labels": ex.get("labels", ""),
        "Scale Label": ex.get("scale_labels", ""),
        "Explanation (PT)": ex.get("scale_label_explanation", ""),
        "Explanation (EN)": translate(ex.get("scale_label_explanation", "")),
    }
    rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)

# Save to Excel or PDF
csv_path = data.metaphors_brazil / "sampled_translations_qwen-3b-brazil-prompted-balanced-1k.csv"
df.to_csv(csv_path, index=False)
print(f"Saved to {csv_path}")
